In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
roi = ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan')
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

In [4]:
# 可视化参数设置
vis_params = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

In [5]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2016-07-01', '2016-07-31') \
#     .map(maskL8sr)

second = ee.Image(landsat_images.toList(landsat_images.size()).get(1))
Map.addLayer(second,vis_params,'second')

In [6]:
print(second.geometry().bounds().getInfo())
Map.addLayer(second.geometry().bounds())

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[114.21294479506497, 29.250772445169307], [116.55148463397202, 29.250772445169307], [116.55148463397202, 31.35132539980697], [114.21294479506497, 31.35132539980697], [114.21294479506497, 29.250772445169307]]]}


In [ ]:
for i in range(3):
    for j in range(3):
        coordinates = ee.Array(second.geometry().bounds().coordinates().get(0))
        leftlon = coordinates.get([0,0]).add(ee.Number(0.82*i))
        leftlat = coordinates.get([0,1]).add(ee.Number(0.72*j))
        rightlon = coordinates.get([0,0]).add(ee.Number(0.82)).add(ee.Number(0.82*i))
        rightlat = coordinates.get([0,1]).add(ee.Number(0.72)).add(ee.Number(0.72*j))
        block = ee.Geometry.Rectangle([leftlon,leftlat,rightlon,rightlat])
        Map.addLayer(block)

In [ ]:
def segmentImage(image):
    imageCol = ee.ImageCollection([])
    for i in range(3):
        for j in range(3):
            coordinates = ee.Array(second.geometry().bounds().coordinates().get(0))
            leftlon = coordinates.get([0,0]).add(ee.Number(0.82*i))
            leftlat = coordinates.get([0,1]).add(ee.Number(0.72*j))
            rightlon = coordinates.get([0,0]).add(ee.Number(0.82)).add(ee.Number(0.82*i))
            rightlat = coordinates.get([0,1]).add(ee.Number(0.72)).add(ee.Number(0.72*j))
            img = image.clip(ee.Geometry.Rectangle([leftlon,leftlat,rightlon,rightlat])).clip(roi)
            
            imageCol = imageCol.merge(ee.ImageCollection(img))
    return imageCol

In [ ]:
def segmentImage(image):
    imageCol = ee.ImageCollection([])
    for i in range(3):
        for j in range(3):
            coordinates = ee.Array(second.geometry().bounds().coordinates().get(0))
            leftlon = coordinates.get([0,0]).add(ee.Number(0.82*i))
            leftlat = coordinates.get([0,1]).add(ee.Number(0.72*j))
            rightlon = coordinates.get([0,0]).add(ee.Number(0.82)).add(ee.Number(0.82*i))
            rightlat = coordinates.get([0,1]).add(ee.Number(0.72)).add(ee.Number(0.72*j))
            img = image.clip(ee.Geometry.Rectangle([leftlon,leftlat,rightlon,rightlat]))
            
            imageCol = imageCol.merge(ee.ImageCollection(img))
    return imageCol

In [ ]:
img = segmentImage(second)
for i in range(img.size().getInfo()):
    num = ee.Image(img.toList(img.size()).get(i))
    Map.addLayer(num,vis_params,"num {}".format(i))

In [ ]:
Map